In [1]:
import numpy as np

from grg_pheno_sim.multi_grg_phenotype import sim_phenotypes_multi_grg
from grg_pheno_sim.model import grg_causal_mutation_model


The following commands only serve the purpose of converting the VCF zip file into GRGs that will be used for phenotype simulation.

In [2]:
%%script bash --out /dev/null
if [ ! -f test-200-samples.grg ]; then
  grg construct --no-maf-flip -p 10 -t 2 ../data/test-200-samples.vcf.gz --out-file test-200-samples.grg
fi

In [3]:
%%script bash --out /dev/null
if [ ! -f test-200-samples_copy.grg ]; then
  grg construct --no-maf-flip -p 10 -t 2 ../data/test-200-samples.vcf.gz --out-file test-200-samples_copy.grg
fi

In [4]:
grg_list = ["test-200-samples.grg", "test-200-samples_copy.grg"]
#this is the list of GRG files to be loaded in 

We will first demonstrate loading all GRG files into RAM and simulating phenotypes. Causal mutations are sampled from each GRG, and the genetic values are obtained for the samples. The combined genetic dataframe is the addition of each GRG's genetic values (for each causal mutation ID). Noise is sampled at the end and added to obtain the phenotypes.

NOTE: It is necessary for each GRG to have the same number of samples.

In [5]:
model_type = "multivariate normal"
means = np.zeros(2)
cov = np.array([[1, 0.25], [0.25, 1]])

model = grg_causal_mutation_model(model_type, mean=means, cov=cov)

num_causal_per_file = 1000

random_seed = 1

normalize_genetic_values_before_noise = False

noise_heritability = [0.33, 0.25]

load_all_into_RAM = True #this parameter decides whether to load all GRGs into RAM together

multi_grg_multi_phenotypes = sim_phenotypes_multi_grg(grg_list, model, num_causal_per_file, random_seed, load_all_ram=load_all_into_RAM, normalize_genetic_values_before_noise=normalize_genetic_values_before_noise, noise_heritability=noise_heritability)


Loaded test-200-samples.grg into RAM
Loaded test-200-samples_copy.grg into RAM
Genetic values for test-200-samples.grg are as follows:
     individual_id  genetic_value  causal_mutation_id
0                0      14.797912                   0
1                0       5.822202                   1
2                1      20.954107                   0
3                1       8.192651                   1
4                2      17.559116                   0
..             ...            ...                 ...
395            197       1.907113                   1
396            198       8.026551                   0
397            198      -4.038713                   1
398            199      28.740603                   0
399            199     -10.363031                   1

[400 rows x 3 columns]
Genetic values for test-200-samples_copy.grg are as follows:
     individual_id  genetic_value  causal_mutation_id
0                0       4.571953                   0
1                0      

In [6]:
multi_grg_multi_phenotypes

,causal_mutation_id,individual_id,genetic_value,environmental_noise,phenotype
0,0,0,-0.430085,0.560838,0.130753
1,1,0,-0.103453,0.135304,0.031851
2,0,1,0.082780,-0.377328,-0.294548
3,1,1,0.174941,-0.498558,-0.323617
4,0,2,-0.060961,0.576355,0.515394
...,...,...,...,...,...
395,1,197,-0.233635,-2.052515,-2.286150
396,0,198,-0.503226,-0.123772,-0.626998
397,1,198,-0.179928,0.063134,-0.116794
398,0,199,-0.055796,0.858229,0.802432


We now perform similar simulations, but by loading the GRGs into RAM sequentially (instead of all together).

In [7]:
model_type = "multivariate normal"
means = np.zeros(2)
cov = np.array([[1, 0.25], [0.25, 1]])

model = grg_causal_mutation_model(model_type, mean=means, cov=cov)

num_causal_per_file = 1000

random_seed = 1

normalize_genetic_values_before_noise = False

noise_heritability = [0.33, 0.25]

load_all_into_RAM = False #this parameter decides whether to load all GRGs into RAM together

multi_grg_multi_seq_phenotypes = sim_phenotypes_multi_grg(grg_list, model, num_causal_per_file, random_seed, load_all_ram=load_all_into_RAM, normalize_genetic_values_before_noise=normalize_genetic_values_before_noise, noise_heritability=noise_heritability)


Loaded test-200-samples.grg into RAM
Genetic values for test-200-samples.grg are as follows:
     individual_id  genetic_value  causal_mutation_id
0                0       1.014481                   0
1                0      11.474887                   1
2                1      -4.051433                   0
3                1      16.344665                   1
4                2     -19.779400                   0
..             ...            ...                 ...
395            197      26.228769                   1
396            198      -5.608502                   0
397            198      33.854848                   1
398            199     -14.088153                   0
399            199       3.224345                   1

[400 rows x 3 columns]
Loaded test-200-samples_copy.grg into RAM
Genetic values for test-200-samples_copy.grg are as follows:
     individual_id  genetic_value  causal_mutation_id
0                0      11.980837                   0
1                0     -

In [8]:
multi_grg_multi_seq_phenotypes

,causal_mutation_id,individual_id,genetic_value,environmental_noise,phenotype
0,0,0,0.722193,0.331943,1.054136
1,1,0,-0.164692,1.061952,0.897260
2,0,1,-0.978688,-0.036144,-1.014832
3,1,1,-0.149369,-0.196450,-0.345819
4,0,2,-0.653737,0.304391,-0.349346
...,...,...,...,...,...
395,1,197,0.262313,1.032835,1.295148
396,0,198,0.542788,-1.167116,-0.624328
397,1,198,0.653400,0.510391,1.163790
398,0,199,-0.284636,0.652633,0.367997
